# Aufgabe-G-WiederholteFragen

## Allgemeines

Eine allgemeine Beschreibung der Laboraufgaben inklusive des Vorgehens, den Bewertungsrichtlinien und der Abgabe finden Sie  <a href="ML-allgemein.ipynb">hier</a>

## Datenquelle


* Laden Sie ihre Daten von http://141.72.190.207/ml_lab/G_fragen herunter
    * Die Daten sind geschützt. 
        * Sie müssen evtl. in einem Netzwerk der DHBW (z.B. WLAN, VPN, ...) angemeldet sein. 
        * Sie können sich auf der Webseite mit dem Benutzernamen dhbw und dem Zugangsnamen ml_2021 anmelden. 
* Die Daten sind in einem anwendungsspezifischen Format gespeichert.
    * Sie finden evtl. Informationen über die Daten in einer "README" Datei. 
    * Finden Sie keine solche Datei sind die Daten selbst erklärend. 
    
    



## Aufgabe

Die Datensammlung enthält Fragen aus einem Forum. Es sind Paare aus Fragen gleichen Inhalts geboten.

Erstellen Sie ein tiefes Neuronales Netzwerk, dass dem Nutzer schon bei der Eingabe einer neuen Frage gleich ähnliche Fragen anbietet. 
* Je mehr Text der Benutzer eingibt, desto präziser sollen die ähnlichen Fragen sein! 




# Lösung

* Beginnen Sie hier mit Ihrer Dokumentation und Implementierung! 

In [8]:
# Fügen Sie ihren Code hier ein!

Vorgehen:
- Finden von unnötigen Wörtern (in, wie usw.)
- Extrahieren der "wertvollen" Informationswerten
- Einteilen der Fragen in Kategorien und Lernen KI anhand der gelieferten Paare
- Überprüfung anhand der Testfragen
- Erstellung einer Eingabemaske, mit Testfragen zur überprüfung der KI

# Einlesen der Daten

## Einlesen der Trainingsdaten

In [46]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras import layers

train_data = pd.read_csv(
    "./fragen/train.csv",
    usecols= ['question1', 'question2', 'is_duplicate'],
    encoding='utf-8'
)

train_data.dropna(inplace=True)
train_data.head()
#for i in range(0, len(train_data)):
#    if type(train_data.at[i, 'question1']) == float or type(train_data.at[i, 'question2']) == float:
#        print(train_data.at[i, 'question2'])
#        print(train_data.at[i, 'question1'])
#        print(i)

,question1,question2,is_duplicate
0,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


## Einlesen der Testdaten

In [47]:
test_data = pd.read_csv(
    "./fragen/test.csv",
    index_col="test_id"
)

test_data.head()


/tmp/ipykernel_4080/1675913220.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  test_data = pd.read_csv(


,question1,question2
test_id,,
0,How does the Surface Pro himself 4 compare wit...,Why did Microsoft choose core m3 and not core ...
1,Should I have a hair transplant at age 24? How...,How much cost does hair transplant require?
2,What but is the best way to send money from Ch...,What you send money to China?
3,Which food not emulsifiers?,What foods fibre?
4,"How ""aberystwyth"" start reading?",How their can I start reading?


# Text preprocessing
According to the tensorflow website

In [48]:
target = train_data.pop('is_duplicate')

ds_raw = tf.data.Dataset.from_tensor_slices((train_data.values, target.values))

for ex in ds_raw.take(3):
    print(ex[0].numpy()[0][:50], ex[0].numpy()[1][:50], ex[1])

b'What is the step by step guide to invest in share ' b'What is the step by step guide to invest in share ' tf.Tensor(0, shape=(), dtype=int64)
b'What is the story of Kohinoor (Koh-i-Noor) Diamond' b'What would happen if the Indian government stole t' tf.Tensor(0, shape=(), dtype=int64)
b'How can I increase the speed of my internet connec' b'How can Internet speed be increased by hacking thr' tf.Tensor(0, shape=(), dtype=int64)


## Unterteilung in Trainings, Validierungs und Testdaten


In [49]:
tf.random.set_seed(1)

ds_length = len(ds_raw)
ds_raw_train = ds_raw.take(round(ds_length*0.8)-1)
ds_raw_train_valid = ds_raw.take(round(ds_length*0.2)-1)

## Create dictionary

In [50]:
from collections import Counter

tokenizer = tfds.deprecated.text.Tokenizer()
token_counts = Counter()

for example in ds_raw_train:
    for i in range(0,2):
        tokens = tokenizer.tokenize(example[0].numpy()[i])
        token_counts.update(tokens)

print("Größe des Wörterbuches: ", len(token_counts))

Größe des Wörterbuches:  99353


## Create a encoder and map the arrays

In [51]:
encoder = tfds.deprecated.text.TokenTextEncoder(token_counts)
encoder.encode('This is a example')

def encode(text_tensor, label):
    #print(text_tensor)
    text0 = text_tensor.numpy()[0]
    text1 = text_tensor.numpy()[1]
    return encoder.encode(text0), encoder.encode(text1), label

def encode_map_fn(text, label):
    return tf.py_function(encode,
                          inp=[text, label],
                          Tout=[tf.int64, tf.int64, tf.int64])

ds_train = ds_raw_train.map(encode_map_fn)
ds_train_valid = ds_raw_train_valid.map(encode_map_fn)
for x in ds_train.take(5):
    print(x[0].shape)

(14,)
(10,)
(14,)
(11,)
(13,)


In [52]:
## Take a small subset
ds_subset = ds_train.take(8)
for example in ds_subset:
    print('Individual Shape:', example[0].shape)

## batching the datasets
ds_batched = ds_subset.padded_batch(
    4, padded_shapes=([-1], [-1], []))

for batch in ds_batched:
    print('Batch Shape:', batch[0].shape)
    print(batch)

Individual Shape: (14,)
Individual Shape: (10,)
Individual Shape: (14,)
Individual Shape: (11,)
Individual Shape: (13,)
Individual Shape: (17,)
Individual Shape: (4,)
Individual Shape: (7,)
Batch Shape: (4, 14)
(<tf.Tensor: shape=(4, 14), dtype=int64, numpy=
array([[ 1,  2,  3,  4,  5,  4,  6,  7,  8,  9, 10, 11,  9, 12],
       [ 1,  2,  3, 13, 14, 15, 16, 17, 18, 19,  0,  0,  0,  0],
       [28, 29, 30, 31,  3, 32, 14, 33, 34, 35, 36, 37, 38, 39],
       [46, 47, 30, 48, 49, 50, 28, 29, 30, 51, 52,  0,  0,  0]])>, <tf.Tensor: shape=(4, 15), dtype=int64, numpy=
array([[ 1,  2,  3,  4,  5,  4,  6,  7,  8,  9, 10, 11,  0,  0,  0],
       [ 1, 20, 21, 22,  3, 23, 24, 25,  3, 15, 16, 17, 18, 26, 27],
       [28, 29, 40, 32, 41, 42,  5, 43, 44, 45,  0,  0,  0,  0,  0],
       [53,  3, 54, 55, 56, 57, 58, 56,  2, 59,  5, 58, 57,  0,  0]])>, <tf.Tensor: shape=(4,), dtype=int64, numpy=array([0, 0, 0, 0])>)
Batch Shape: (4, 17)
(<tf.Tensor: shape=(4, 17), dtype=int64, numpy=
array([[ 60,  61, 

# Batch the data


In [57]:
train_data = ds_train.padded_batch( 32, padded_shapes=([-1],[-1],[]))

valid_data = ds_train_valid.padded_batch( 32, padded_shapes=([-1],[-1],[]))

#test_data  = ds_test.padded_batch ( 32, padded_shapes=([-1],[-1],[]))

Individual Shape: tf.Tensor([0 0 0 0 0 1 0 1 0 0 0 1 1 1 0 1 1 0 1 0 1 0 0 0 0 0 0 0 0 1 0 1], shape=(32,), dtype=int64)
Individual Shape: tf.Tensor([1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 1 1 1 0 1 0 0 0 0 1 0 0 0 1 0], shape=(32,), dtype=int64)
Individual Shape: tf.Tensor([0 1 1 1 0 0 0 1 1 1 1 0 0 0 0 1 0 0 0 0 1 1 1 0 1 0 0 0 1 1 0 1], shape=(32,), dtype=int64)
Individual Shape: tf.Tensor([0 0 0 0 1 0 0 0 1 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1 0 0 1 0 1], shape=(32,), dtype=int64)
Individual Shape: tf.Tensor([0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 1 0 0 0 1 0 1 1], shape=(32,), dtype=int64)
Individual Shape: tf.Tensor([1 0 0 1 0 1 0 0 1 0 0 0 0 1 0 1 1 0 1 1 1 0 1 0 0 1 0 0 1 1 1 1], shape=(32,), dtype=int64)
Individual Shape: tf.Tensor([0 1 1 0 0 1 1 1 1 0 0 1 0 0 0 0 0 1 1 0 0 0 0 1 1 0 0 1 1 1 0 0], shape=(32,), dtype=int64)
Individual Shape: tf.Tensor([1 0 1 0 0 1 0 0 0 0 0 1 1 0 1 0 0 0 1 1 1 0 1 0 0 1 1 1 0 1 0 1], shape=(32,), dtype=int64)


# Add the trainings-modell

In [43]:
from keras.layers import Embedding, SimpleRNN, Dense

size_dic = len(token_counts) + 2
print("Größe Wörterbuch + 2", size_dic)

model = tf.keras.Sequential()
model.add(Embedding(size_dic, output_dim = 32))
model.add(SimpleRNN(32, return_sequences=True))
model.add(SimpleRNN(32))
model.add(Dense(1))
model.summary()

Größe Wörterbuch + 2 99355
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 32)          3179360   
                                                                 
 simple_rnn_2 (SimpleRNN)    (None, None, 32)          2080      
                                                                 
 simple_rnn_3 (SimpleRNN)    (None, 32)                2080      
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 3,183,553
Trainable params: 3,183,553
Non-trainable params: 0
_________________________________________________________________


In [49]:
model.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
              metrics=['accuracy'])
history = model.fit(train_data, validation_data=valid_data, epochs=10)

Epoch 1/10
10108/10108 [==============================] - 355s 35ms/step - loss: -4745.5591 - accuracy: 0.0133 - val_loss: -4192.2656 - val_accuracy: 0.0129
Epoch 2/10
10108/10108 [==============================] - 360s 36ms/step - loss: -4746.2764 - accuracy: 0.0130 - val_loss: -4192.2656 - val_accuracy: 0.0129
Epoch 3/10
10108/10108 [==============================] - 363s 36ms/step - loss: -4746.2764 - accuracy: 0.0130 - val_loss: -4192.2656 - val_accuracy: 0.0129
Epoch 4/10
10108/10108 [==============================] - 363s 36ms/step - loss: -4746.2764 - accuracy: 0.0130 - val_loss: -4192.2656 - val_accuracy: 0.0129
Epoch 5/10
10108/10108 [==============================] - 363s 36ms/step - loss: -4746.2764 - accuracy: 0.0130 - val_loss: -4192.2656 - val_accuracy: 0.0129
Epoch 6/10
10108/10108 [==============================] - 364s 36ms/step - loss: -4746.2764 - accuracy: 0.0130 - val_loss: -4192.2656 - val_accuracy: 0.0129
Epoch 7/10
10108/10108 [==============================] - 

In [14]:
import os
model = tf.keras.models.load_model('./models/rnn.h5')
test_results = model.evaluate(train_data)
print(f'Test acc : {test_results}')
if not os.path.exists('models'):
    os.mkdir('models')


model.save('models/rnn.h5')

ValueError: in user code:

    File "/home/maxh/Gruppe_G_Fragen/venv/lib/python3.9/site-packages/keras/engine/training.py", line 1801, in predict_function  *
        return step_function(self, iterator)
    File "/home/maxh/Gruppe_G_Fragen/venv/lib/python3.9/site-packages/keras/engine/training.py", line 1790, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/maxh/Gruppe_G_Fragen/venv/lib/python3.9/site-packages/keras/engine/training.py", line 1783, in run_step  **
        outputs = model.predict_step(data)
    File "/home/maxh/Gruppe_G_Fragen/venv/lib/python3.9/site-packages/keras/engine/training.py", line 1751, in predict_step
        return self(x, training=False)
    File "/home/maxh/Gruppe_G_Fragen/venv/lib/python3.9/site-packages/keras/utils/traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/home/maxh/Gruppe_G_Fragen/venv/lib/python3.9/site-packages/keras/engine/

In [45]:
model = tf.keras.models.load_model('./models/rnn.h5')
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 32)          3179360   
                                                                 
 simple_rnn_2 (SimpleRNN)    (None, None, 32)          2080      
                                                                 
 simple_rnn_3 (SimpleRNN)    (None, 32)                2080      
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 3,183,553
Trainable params: 3,183,553
Non-trainable params: 0
_________________________________________________________________


ValueError: Layer "sequential_2" expects 1 input(s), but it received 3 input tensors. Inputs received: [<tf.Tensor: shape=(32, 29), dtype=int64, numpy=
array([[  1,   2,   3,   4,   5,   4,   6,   7,   8,   9,  10,  11,   9,
         12,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0],
       [  1,   2,   3,  13,  14,  15,  16,  17,  18,  19,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0],
       [ 28,  29,  30,  31,   3,  32,  14,  33,  34,  35,  36,  37,  38,
         39,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0],
       [ 46,  47,  30,  48,  49,  50,  28,  29,  30,  51,  52,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0],
       [ 60,  61,  62,   9,  63,  64,  65,  66,  67,  68,  69,  70,  71,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0]